In [1]:
from PIL import Image
import scipy.misc
import os

from __future__ import print_function

import scipy.io as spio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import keras
from keras.models import model_from_json

patch_size = 50

def evaluateScore(x_): 
    # modify x_test y_test(GT) by sliding a window in test image to obtain x_test
    x_test = np.zeros((1,patch_size, patch_size, 1), dtype=np.float)
    x_test[0,:,:,0] = x_;
    y_test = np.zeros((1,2), dtype=np.float)
    
    # evaluate loaded model on test data
    scores = loaded_model.evaluate(x_test, y_test, verbose=0)  
    return scores[0]

Using TensorFlow backend.


In [ ]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
loaded_model.compile(loss='binary_crossentropy', 
                     optimizer='rmsprop', 
                     metrics=['accuracy'])

#load one test image
image = Image.open("test_img1.png")#.convert('L')
#image.show()

#1.create a RGB image (same size of original image) to show the visualization 
#2.make the test_box GREEN, if the test_box is detected positive, else RED

RED = Image.new('RGBA', (patch_size, patch_size), (255,0,0,128))
GREEN = Image.new('RGBA', (patch_size, patch_size), (0,255,0,128))
for i in range(0, image.size[0], patch_size):
    for j in range(0, image.size[1], patch_size):            
        box = image.crop((i, j, i + patch_size, j + patch_size))                
        x = i
        y = j
        w = patch_size
        h = patch_size
        
        data = np.asarray(box)  
        numpy_sum = np.sum(np.sum(data, axis=0), axis=0)
        print('row:', i, ' col:', j, '\t', 'numpy_sum:',numpy_sum)
        
        # evaluate this patch
        score = evaluateScore(data)        
        print('Evaluate score:=', score)
       
        ## save masked image
        name = ('test/ImgWithPatch_i{}_j{}.png').format(i, j)
        print(name, '\n')
        result = image.copy();
        result = result.convert('RGBA')
        break
#         if score < 4:
#             result.paste(RED, (i, j, i + patch_size, j + patch_size)) 
#         else:
#             result.paste(GREEN, (i, j, i + patch_size, j + patch_size))
#         result.save(name, "PNG")
print(max(score, key=score.get))
